### Import Libraries

In [186]:
# Import twitter dependencies
import tweepy
from config import *

In [187]:
# Import transformation dependencies
import pandas as pd
from datetime import datetime, date
import json
import re 

In [188]:
# For sentiment and subjectivity analysis
from textblob import TextBlob

# Tweepy Setup

In [189]:
# Tweepy Setup
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [248]:
# Tweepy search parameters
search_terms = ['covid+vaccine OR covid-19+vaccine OR pfizer OR pfizer-biontech OR moderna OR astrazeneca OR astra+zeneca -filter:retweets']
end_date = date(2021,3,11)
max_id ='1370151964881862656'
limit = 1500 # this is my 15 minute limit :'( breaks if this becomes 1000

# Save files
output_csv ='csvs/tw40.csv'
output_json ='json/raw40.json'

# Functions

Collect and save data

In [249]:
# Pulls data from api and returns 2 lists:
    # one which is parsed for selected variables
    # one with all the raw data

data = []
raw = []
tweets = tweepy.Cursor(api.search, location='canada', max_id = max_id, place='canada', lang ='en', q=search_terms, tweet_mode='extended').items(limit)
for t in tweets: 
    tweet ={
        'created' :t.created_at,
        'id_str' : t.id_str,
        'text' : t.full_text,
        'user_id' :t.user.id,
        'screen_name': t.user.screen_name,
        'location': t.user.location,
        'followers_count': t.user.followers_count,
        'user_favourites_count': t.user.favourites_count,
        'time_zone' :t.user.time_zone,
        'geo_enabled' :t.user.geo_enabled,
        'verified' :t.user.verified,
        'status_count': t.user.statuses_count,
        'geo':t.geo,
        'coords':t.coordinates,
        'retweet_count': t.retweet_count,
        'tweet_favourite_count' : t.favorite_count
    }
    # add sorted data to one list 
    data.append(tweet)
    # keep raw data in case I realize later that I've removed something important 
    raw.append(t)
print('done!')

done!


In [250]:
# Add parsed data to dataframe, convert to csv
tweets_df = pd.DataFrame(data)

# Save raw data to json 
with open(output_json, 'w') as file:
    for resp in raw:
        jso = json.dumps(resp._json,sort_keys=True, indent=3)
        file.write(jso)
tweets_df.tail(3)

,created,id_str,text,user_id,screen_name,location,followers_count,user_favourites_count,time_zone,geo_enabled,verified,status_count,geo,coords,retweet_count,tweet_favourite_count
1497,2021-03-11 23:10:17,1370150112794718213,RT @donkamion78: 12 Prominent scientists and d...,1134041472989323265,ArcheryAsgard,(United) Kingdom,233,13244,None,False,False,5376,None,None,192,0
1498,2021-03-11 23:10:17,1370150112773730307,RT @DamionMitch: Professor Denise Edemire Shea...,2518005376,ahongry,Jamaica,54,9740,None,False,False,17679,None,None,4,0
1499,2021-03-11 23:10:17,1370150112383631360,I hope the Covid vaccine does rewrite my DNA a...,373978358,_KyleTurner,"Brooklyn, NY",545,767,None,False,False,2040,None,None,1,10


### Clean tweets

In [251]:
# not working when I put these into a function
tweets_df = tweets_df.drop_duplicates(subset=['id_str'])
tweets_df = tweets_df.dropna(subset=['id_str', 'screen_name'])

In [252]:
# Makes tweets more readable 
def CleanTweets(txt): 
    # remove @ username 
    # txt = re.sub(r'@[A-Za-z0-9)]+','', txt)

    # remove @ from username
    txt = txt.replace('@','')    
    
    # remove RT    
    txt = re.sub(r'RT[\s]+','',txt)
    
    # remove # but leave txt
    txt = re.sub(r'#','', txt)
    
    # remove hyperlinks
    txt = re.sub(r'https?:\/\/\S+', '', txt)
    return txt

In [253]:
# Clean the text field 
tweets_df['text'] = tweets_df['text'].apply(CleanTweets)

In [254]:
# to get subjectivity 
def RateSubjectivity(txt):
    return TextBlob(txt).sentiment.subjectivity

In [255]:
# to get polarity
def RatePolarity(txt):
    return TextBlob(txt).sentiment.polarity

In [256]:
# add worded sentiment based on polarity score
def GetSentiment(num):
    if num < 0: 
        return 'negative' 
    elif num == 0:
        return 'neutral'
    else:
        return 'positive'

In [257]:
# Create Columns
def RateTweets(df):
    df['Subjectivity'] = df['text'].apply(RateSubjectivity)
    df['Polarity'] = df['text'].apply(RatePolarity)
    df['Sentiment'] = df['Polarity'].apply(GetSentiment)
    return df 

In [258]:
tweets_df = RateTweets(tweets_df)
tweets_df.to_csv(output_csv, index=False, encoding='UTF-8')
tweets_df.tail(3)

,created,id_str,text,user_id,screen_name,location,followers_count,user_favourites_count,time_zone,geo_enabled,verified,status_count,geo,coords,retweet_count,tweet_favourite_count,Subjectivity,Polarity,Sentiment
1497,2021-03-11 23:10:17,1370150112794718213,donkamion78: 12 Prominent scientists and docto...,1134041472989323265,ArcheryAsgard,(United) Kingdom,233,13244,None,False,False,5376,None,None,192,0,0.555556,0.277778,positive
1498,2021-03-11 23:10:17,1370150112773730307,DamionMitch: Professor Denise Edemire Shearer ...,2518005376,ahongry,Jamaica,54,9740,None,False,False,17679,None,None,4,0,0.500000,0.416667,positive
1499,2021-03-11 23:10:17,1370150112383631360,I hope the Covid vaccine does rewrite my DNA a...,373978358,_KyleTurner,"Brooklyn, NY",545,767,None,False,False,2040,None,None,1,10,0.000000,0.000000,neutral


# SQLAlchemy
### Connecting to a certain other host 

In [5]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [12]:
engine = create_engine(conn, echo=False)

In [13]:
Base = automap_base()
Base.prepare(engine, reflect=True)  

In [14]:
# Test connection
# it works!!!
# not anymore
# kill kernal if conn is wrong

Base.classes.keys()

['tw2']

### Don't run yet
Clean up your tweets! 

In [9]:
_df = pd.read_csv('csvs/tw2.csv')
_df.to_sql(name='tw2', con=engine, if_exists='append', index=True)
print('added!')

added!


In [ ]:
# session things
session = Session(engine)
session.close()
